# 連続系の代表的な伝達関数の離散化
© 2020 @RR_Inyo

LTspiceおよび制御用自作ライブラリContrailleに実装するため，SymPy用いて，連続系における下記の伝達関数を双一次変換（Bilinear transform, Tustin transformとも）によって離散化しよう。

|要素名|連続系における伝達関数|
|:----|:----|
|積分要素|$G_{1}(s) = \displaystyle \frac{1}{s}$|
|微分要素|$G_{2}(s) = s$|
|1次LPF (1)|$G_{3}(s) = \displaystyle \frac{1}{1 + sT}$|
|1次LPF (2)|$G_{4}(s) = \displaystyle \frac{\omega_{n}}{s + \omega_{n}}$|
|1次HPF (1)|$G_{5}(s) = \displaystyle \frac{s T}{1 + s T}$|
|1次HPF (2)|$G_{6}(s) = \displaystyle \frac{s}{s + \omega_{n}}$|
|2次LPF|$G_{7}(s) = \displaystyle \frac{\omega_{n}^{2}}{s^2 + 2 \zeta \omega_{n} s + \omega_{n}^{2}}$|
|2次HPF|$G_{8}(s) = \displaystyle \frac{s^{2}}{s^{2} + 2 \zeta \omega_{n} s + \omega_{n}^{2}}$|
|2次BPF|$G_{9}(s) = \displaystyle \frac{2 \zeta \omega_{n} s}{s^{2} + 2 \zeta \omega_{n} s + \omega_{n}^{2}}$|
|2次BEF|$G_{10}(s) = \displaystyle \frac{s^{2} + \omega_{n}^{2}}{s^{2} + 2 \zeta \omega_{n} s + \omega_{n}^{2}}$|

ここで，サンプリング周期を$T_{s}$とすると，双一次変換は次式で表される。
$$
s = \frac{2}{T_{s}} \frac{z - 1}{z + 1} = \frac{2}{T_{s}} \frac{1 - z^{-1}}{1 + z^{-1}}
$$

In [1]:
# 準備
import sympy as sp

In [2]:
# シンボルの定義
s = sp.symbols('s')
z, T_s = sp.symbols('z T_s')
T, omega_n, zeta = sp.symbols('T omega_n zeta')

# 各伝達関数の定義
G1 = 1 / s # 積分要素
G2 = s # 微分要素
G3 = 1 / (1 + s * T) # 1次LPF(1)
G4 = omega_n / (s + omega_n) # 1次LPF(2)
G5 = s * T / (1 + s * T) # 1次HPF(1)
G6 = s / (s + omega_n) # 1次HPF(2)
G7 = omega_n**2 / (s**2 + 2 * zeta * omega_n * s + omega_n**2) # 2次LPF
G8 = s**2 / (s**2 + 2 * zeta * omega_n * s + omega_n**2) # 2次HPF
G9 = 2 * zeta * omega_n * s / (s**2 + 2 * zeta * omega_n * s + omega_n**2) # 2次BPF
G10 = (s**2 + omega_n**2) / (s**2 + 2 * zeta * omega_n * s + omega_n**2) # 2次BEF

In [3]:
# 双一次変換（試行錯誤の急増関数）
def tustin(G):
    G_d = sp.simplify(G.subs(s, 2 / T_s * (z - 1) / (z + 1))) # 双一次変換
    G_d = sp.collect(sp.expand(sp.numer(G_d)), z) / sp.collect(sp.expand(sp.denom(G_d)), z) # 分母分子をzの多項式に整理
    return G_d

def num_simp(G_d):
    num = sp.collect(sp.expand(sp.numer(G_d) / (omega_n**2 * T_s**2)), z)
    return num

def den_simp(G_d):
    den = sp.collect(sp.expand(sp.denom(G_d) / (omega_n**2 * T_s**2)), z)
    return den

## 積分要素と微分要素

In [4]:
# 積分要素
G1_d = tustin(G1)
G1_d

(T_s*z + T_s)/(2*z - 2)

In [5]:
# 微分要素
G2_d = tustin(G2)
G2_d

(2*z - 2)/(T_s*z + T_s)

以上より，$G_{1}(s)$と$G_{2}(s)$を離散化した$G_{1d}(z)$と$G_{2d}(z)$は
$$
\begin{align}
G_{1d}(z) &= \frac{T_{s} z + T_{s}}{2 z - 2} = \frac{T_{s} + T_{s} z^{-1}}{2 - 2 z^{-1}} = \frac{\frac{T_{s}}{2} + \frac{T_{s}}{2} z^{-1}}{1 - z^{-1}} \\[5pt]
G_{2d}(z) &= \frac{2 z - 2}{T_{s} z + T_{s}} = \frac{2 - 2 z^{-1}}{T_{s} + T_{s} z^{-1}} = \frac{\frac{2}{T_{s}}- \frac{2}{T_{s}} z^{-1}}{1 + z^{-1}}
\end{align}
$$
となる。

## 1次系

In [6]:
# 1次LPF(1)
G3_d = tustin(G3)
G3_d

(T_s*z + T_s)/(-2*T + T_s + z*(2*T + T_s))

In [7]:
# 1次LPF(2)
G4_d = tustin(G4)
G4_d

(T_s*omega_n*z + T_s*omega_n)/(T_s*omega_n + z*(T_s*omega_n + 2) - 2)

In [8]:
# 1次HPF(1)
G5_d = tustin(G5)
G5_d

(2*T*z - 2*T)/(-2*T + T_s + z*(2*T + T_s))

In [9]:
# 1次HPF(2)
G6_d = tustin(G6)
G6_d

(2*z - 2)/(T_s*omega_n + z*(T_s*omega_n + 2) - 2)

以上より，以上より，$G_{3}(s)$ ~ $G_{6}(s)$を離散化した$G_{3d}(z)$ ~ $G_{6d}(z)$は
$$
\begin{align}
G_{3d}(z) &= \frac{T_{s} z + T_{s}}{(T_{s} + 2 T) z + T_{s} - 2 T} = \frac{T_{s} + T_{s} z^{-1}}{(T_{s} + 2 T) + (T_{s} - 2 T) z^{-1}} = \frac{\frac{T_{s}}{T_{s} + 2 T} + \frac{T_{s}}{T_{s} + 2 T} z^{-1}}{1 + \frac{T_{s} - 2 T}{T_{s} + 2 T} z^{-1}} \\[5pt]
G_{4d}(z) &= \frac{\omega_{n} T_{s} z + \omega_{n} T_{s}}{(\omega T_{s} + 2) z + \omega_{n} T_{s} - 2} = \frac{\omega_{n} T_{s} + \omega_{n} T_{s} z^{-1}}{\omega T_{s} + 2 + (\omega_{n} T_{s} - 2) z^{-1}} = \frac{\frac{\omega_{n} T_{s}}{\omega T_{s} + 2} + \frac{\omega_{n} T_{s}}{\omega T_{s} + 2} z^{-1}}{1 + \frac{\omega_{n} T_{s} - 2}{\omega T_{s} + 2} z^{-1}} \\[5pt]
G_{5d}(z) &= \frac{2 T z - 2 T}{(T_{s} + 2 T) z + T_{s} - 2 T} = \frac{2 T - 2 T z^{-1}}{T_{s} + 2 T + (T_{s} - 2 T) z^{-1}} = \frac{\frac{2 T}{T_{s} + 2 T} - \frac{2 T}{T_{s} + 2 T} z^{-1}}{1 + \frac{T_{s} - 2 T}{T_{s} + 2 T} z^{-1}} \\[5pt]
G_{6d}(z) &= \frac{2 z - 2}{(\omega T_{s} + 2) z + \omega_{n} T_{s} - 2} = \frac{2 - 2 z^{-1}}{\omega T_{s} + 2 + (\omega_{n} T_{s} - 2) z^{-1}} = \frac{\frac{2}{\omega T_{s} + 2} - \frac{2}{\omega T_{s} + 2} z^{-1}}{1 + \frac{\omega_{n} T_{s} - 2}{\omega T_{s} + 2} z^{-1}}
\end{align}
$$
となる。

## 2次系

In [10]:
# 2次系LPF
G7_d = tustin(G7)
G7_d

(T_s**2*omega_n**2*z**2 + 2*T_s**2*omega_n**2*z + T_s**2*omega_n**2)/(T_s**2*omega_n**2 - 4*T_s*omega_n*zeta + z**2*(T_s**2*omega_n**2 + 4*T_s*omega_n*zeta + 4) + z*(2*T_s**2*omega_n**2 - 8) + 4)

In [11]:
## G7_dの分子を整理
num_simp(G7_d)

z**2 + 2*z + 1

In [12]:
## G7_dの分母を整理
den_simp(G7_d)

z**2*(1 + 4*zeta/(T_s*omega_n) + 4/(T_s**2*omega_n**2)) + z*(2 - 8/(T_s**2*omega_n**2)) + 1 - 4*zeta/(T_s*omega_n) + 4/(T_s**2*omega_n**2)

In [13]:
# 2次系HPF
G8_d = tustin(G8)
G8_d

(4*z**2 - 8*z + 4)/(T_s**2*omega_n**2 - 4*T_s*omega_n*zeta + z**2*(T_s**2*omega_n**2 + 4*T_s*omega_n*zeta + 4) + z*(2*T_s**2*omega_n**2 - 8) + 4)

In [14]:
## G8_dの分子を整理
num_simp(G8_d)

4*z**2/(T_s**2*omega_n**2) - 8*z/(T_s**2*omega_n**2) + 4/(T_s**2*omega_n**2)

In [15]:
## G8_dの分母を整理
den_simp(G8_d)

z**2*(1 + 4*zeta/(T_s*omega_n) + 4/(T_s**2*omega_n**2)) + z*(2 - 8/(T_s**2*omega_n**2)) + 1 - 4*zeta/(T_s*omega_n) + 4/(T_s**2*omega_n**2)

In [16]:
# 2次系BPF
G9_d = tustin(G9)
G9_d

(4*T_s*omega_n*z**2*zeta - 4*T_s*omega_n*zeta)/(T_s**2*omega_n**2 - 4*T_s*omega_n*zeta + z**2*(T_s**2*omega_n**2 + 4*T_s*omega_n*zeta + 4) + z*(2*T_s**2*omega_n**2 - 8) + 4)

In [17]:
## G9_dの分子を整理
num_simp(G9_d)

4*z**2*zeta/(T_s*omega_n) - 4*zeta/(T_s*omega_n)

In [18]:
## G9_dの分母を整理
den_simp(G9_d)

z**2*(1 + 4*zeta/(T_s*omega_n) + 4/(T_s**2*omega_n**2)) + z*(2 - 8/(T_s**2*omega_n**2)) + 1 - 4*zeta/(T_s*omega_n) + 4/(T_s**2*omega_n**2)

In [19]:
# 2次系BEF
G10_d = tustin(G10)
G10_d

(T_s**2*omega_n**2 + z**2*(T_s**2*omega_n**2 + 4) + z*(2*T_s**2*omega_n**2 - 8) + 4)/(T_s**2*omega_n**2 - 4*T_s*omega_n*zeta + z**2*(T_s**2*omega_n**2 + 4*T_s*omega_n*zeta + 4) + z*(2*T_s**2*omega_n**2 - 8) + 4)

In [20]:
## G10_dの分子を整理
sp.collect(sp.expand(num_simp(G10_d)), z)

z**2*(1 + 4/(T_s**2*omega_n**2)) + z*(2 - 8/(T_s**2*omega_n**2)) + 1 + 4/(T_s**2*omega_n**2)

In [21]:
## G10_dの分母を整理
den_simp(G10_d)

z**2*(1 + 4*zeta/(T_s*omega_n) + 4/(T_s**2*omega_n**2)) + z*(2 - 8/(T_s**2*omega_n**2)) + 1 - 4*zeta/(T_s*omega_n) + 4/(T_s**2*omega_n**2)

以上より，$G_{7d}(z)$ ~ $G_{10d}(z)$の分母$D(z)$はすべて
$$
D(z) = \left ( 1 + \frac{4 \zeta}{\omega_{n} T_{s}} + \frac{4}{\omega_{n}^{2} T_{s}^{2}} \right ) z^{2} + 2 \left (1 - \frac{4}{\omega_{n}^{2} T_{s}^{2}} \right ) z + 1 - \frac{4 \zeta}{\omega_{n} T_{s}} + \frac{4}{\omega_{n}^{2} T_{s}^{2}}
$$
である。ここで，$M$と$N$をそれぞれ
$$
\begin{align}
M = \frac{4 \zeta}{\omega_{n} T_{s}} \\[5pt]
N = \frac{4}{\omega_{n}^{2} T_{s}^{2}}
\end{align}
$$
と定義すれば，$D(z)$は
$$
D(z) = (1 + M + N) z^{2} + 2 (1 - N) z^{1} + (1 - M + N) z^{0}
$$
と表現できる。また，上記のSymPyによる計算から，$G_{7}(s)$ ~ $G_{10}(s)$の分子も$M$，$N$を用いて書けることが判る。

以上を踏まえて，$G_{7d}(z)$ ~ $G_{10d}(z)$を$M$，$N$を用いて表す。
$$
\begin{align}
G_{7d}(z) &= \frac{z^{2} + 2 z + 1}{(1 + M + N) z^{2} + 2 (1 - N) z^{1} + 1 - M + N} = \frac{\frac{1}{1 + M + N} + \frac{2}{1 + M + N} z^{-1} + \frac{1}{1 + M + N} z^{-2}}{1 + \frac{2 (1 - N)}{1 + M + N} z^{-1} + \frac{1 - M + N}{1 + M + N} z^{-2}} \\[5pt]
G_{8d}(z) &= \frac{N z^{2} - 2 N z + N}{(1 + M + N) z^{2} + 2 (1 - N) z^{1} + 1 - M + N} = \frac{\frac{N}{1 + M + N} - \frac{2 N}{1 + M + N} z^{-1} + \frac{N}{1 + M + N} z^{-2}}{1 + \frac{2 (1 - N)}{1 + M + N} z^{-1} + \frac{1 - M + N}{1 + M + N} z^{-2}} \\[5pt]
G_{9d}(z) &= \frac{M z^{2} - M}{(1 + M + N) z^{2} + 2 (1 - N) z^{1} + 1 - M + N} = \frac{\frac{M}{1 + M + N} -
\frac{M}{1 + M + N} z^{-2}}{1 + \frac{2 (1 - N)}{1 + M + N} z^{-1} + \frac{1 - M + N}{1 + M + N} z^{-2}} \\[5pt]
G_{10d}(z) &= \frac{(1 + N) z^{2} + 2 (1 - N) z + 1 + N}{(1 + M + N) z^{2} + 2 (1 - N) z^{1} + 1 - M + N} = \frac{\frac{1 + N}{1 + M + N} + \frac{2 (1 - N)}{1 + M + N} z^{-1} + \frac{1 + N}{1 + M + N}}{1 + \frac{2 (1 - N)}{1 + M + N} z^{-1} + \frac{1 - M + N}{1 + M + N} z^{-2}}
\end{align}
$$

## まとめ
連続系の伝達関数$G_{1}(s)$ ~ $G_{10}(s)$と，双一次変換によって得た離散系の伝達関数$G_{1d}(z)$ ~ $G_{10d}$の対応を一覧表にまとめると，下表を得る。

|要素名|連続系における伝達関数|離散系における伝達関数|
|:----|:----|:----|
|積分要素|$G_{1}(s) = \displaystyle \frac{1}{s}$|$G_{1d}(z) = \displaystyle \frac{\frac{T_{s}}{2} + \frac{T_{s}}{2} z^{-1}}{1 - z^{-1}}$|
|微分要素|$G_{2}(s) = s$|$G_{2d}(z) = \displaystyle \frac{\frac{2}{T_{s}}- \frac{2}{T_{s}} z^{-1}}{1 + z^{-1}}$|
|1次LPF (1)|$G_{3}(s) = \displaystyle \frac{1}{1 + sT}$|$G_{3d}(z) = \displaystyle \frac{\frac{T_{s}}{T_{s} + 2 T} + \frac{T_{s}}{T_{s} + 2 T} z^{-1}}{1 + \frac{T_{s} - 2 T}{T_{s} + 2 T} z^{-1}}$|
|1次LPF (2)|$G_{4}(s) = \displaystyle \frac{\omega_{n}}{s + \omega_{n}}$|$G_{4d}(z) = \displaystyle \frac{\frac{\omega_{n} T_{s}}{\omega T_{s} + 2} + \frac{\omega_{n} T_{s}}{\omega T_{s} + 2} z^{-1}}{1 + \frac{\omega_{n} T_{s} - 2}{\omega T_{s} + 2} z^{-1}}$|
|1次HPF (1)|$G_{5}(s) = \displaystyle \frac{s T}{1 + s T}$|$G_{5d}(z) = \displaystyle \frac{\frac{2 T}{T_{s} + 2 T} - \frac{2 T}{T_{s} + 2 T} z^{-1}}{1 + \frac{T_{s} - 2 T}{T_{s} + 2 T} z^{-1}}$|
|1次HPF (2)|$G_{6}(s) = \displaystyle \frac{s}{s + \omega_{n}}$|$G_{6d}(z) = \displaystyle \frac{\frac{2}{\omega T_{s} + 2} - \frac{2}{\omega T_{s} + 2} z^{-1}}{1 + \frac{\omega_{n} T_{s} - 2}{\omega T_{s} + 2} z^{-1}}$|
|2次LPF|$G_{7}(s) = \displaystyle \frac{\omega_{n}^{2}}{s^2 + s \zeta \omega_{n} s + \omega_{n}^{2}}$|$G_{7d}(z) = \displaystyle \frac{\frac{1}{1 + M + N} + \frac{2}{1 + M + N} z^{-1} + \frac{1}{1 + M + N} z^{-2}}{1 + \frac{2 (1 - N)}{1 + M + N} z^{-1} + \frac{1 - M + N}{1 + M + N} z^{-2}}$|
|2次HPF|$G_{8}(s) = \displaystyle \frac{s^{2}}{s^{2} + s \zeta \omega_{n} s + \omega_{n}^{2}}$|$G_{8d}(z) = \displaystyle \frac{\frac{N}{1 + M + N} - \frac{2 N}{1 + M + N} z^{-1} + \frac{N}{1 + M + N} z^{-2}}{1 + \frac{2 (1 - N)}{1 + M + N} z^{-1} + \frac{1 - M + N}{1 + M + N} z^{-2}}$|
|2次BPF|$G_{9}(s) = \displaystyle \frac{2 \zeta \omega_{n} s}{s^{2} + s \zeta \omega_{n} s + \omega_{n}^{2}}$|$G_{9d}(z) = \displaystyle \frac{\frac{M}{1 + M + N} - \frac{M}{1 + M + N} z^{-2}}{1 + \frac{2 (1 - N)}{1 + M + N} z^{-1} + \frac{1 - M + N}{1 + M + N} z^{-2}}$|
|2次BEF|$G_{10}(s) = \displaystyle \frac{s^{2} + \omega_{n}^{2}}{s^{2} + s \zeta \omega_{n} s + \omega_{n}^{2}}$|$G_{10d} = \displaystyle \frac{\frac{1 + N}{1 + M + N} + \frac{2 (1 - N)}{1 + M + N} z^{-1} + \frac{1 + N}{1 + M + N}}{1 + \frac{2 (1 - N)}{1 + M + N} z^{-1} + \frac{1 - M + N}{1 + M + N} z^{-2}}$|

ただし，2次系に関して，
$$
\begin{align}
M = \frac{4 \zeta}{\omega_{n} T_{s}} \\[5pt]
N = \frac{4}{\omega_{n}^{2} T_{s}^{2}}
\end{align}
$$
である。